# Importing The Libraries

In [ ]:
%%capture
# Installs Unsloth, Xformers (Flash Attention) and all other packages!
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.26" trl peft accelerate bitsandbytes
!pip install -U autogenstudio autogen

In [ ]:
import os
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig
from huggingface_hub import login
import autogen
from autogen import AssistantAgent, UserProxyAgent
from types import SimpleNamespace


In [ ]:
login(os.environ.get("HF_API_KEY"))

## Load Base Models

In [ ]:
base_model_localizer = AutoModelForCausalLM.from_pretrained(
    "meta-llama/CodeLlama-13b-Instruct-hf",
    device_map = "auto",
    load_in_4bit = True,
)
base_tokenizer_localizer = AutoTokenizer.from_pretrained("meta-llama/CodeLlama-13b-Instruct-hf")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/589 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors.index.json:   0%|          | 0.00/31.4k [00:00<?, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

...

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.59k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

In [ ]:
base_model_codefixer = AutoModelForCausalLM.from_pretrained(
    "meta-llama/CodeLlama-34b-Instruct-hf",
    device_map = "auto",
    load_in_4bit = True,
)
base_tokenizer_codefixer = AutoTokenizer.from_pretrained("meta-llama/CodeLlama-13b-Instruct-hf")

config.json:   0%|          | 0.00/588 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors.index.json:   0%|          | 0.00/37.6k [00:00<?, ?B/s]

Fetching 7 files:   0%|          | 0/7 [00:00<?, ?it/s]

model-00001-of-00007.safetensors:   0%|          | 0.00/9.85G [00:00<?, ?B/s]

...

model-00004-of-00007.safetensors:   0%|          | 0.00/9.69G [00:00<?, ?B/s]

model-00003-of-00007.safetensors:   0%|          | 0.00/9.69G [00:00<?, ?B/s]

model-00006-of-00007.safetensors:   0%|          | 0.00/9.69G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [ ]:
base_tokenizer_localizer.pad_token = base_tokenizer_localizer.eos_token
base_tokenizer_codefixer.pad_token = base_tokenizer_codefixer.eos_token

In [ ]:
buggy_code = """
public String readLine() throws IOException {
	// with these functions, we consider a line as always terminated by CRLF
	serverByteBuffer sb = new serverByteBuffer();
	int c;
	while (true) {
	    c = read();
	    if (c < 0) {
		if (sb.length() == 0) return null; else return sb.toString();
	    }
	    if (c == cr) continue;
	    if (c == lf) return sb.toString();
	    sb.append((byte) c);
	}
    }
"""
bug_info = """
{'bug_type': 'RESOURCE_LEAK',
 'qualifier': 'resource of type `de.anomic.server.serverByteBuffer` acquired to `sb` by call to `new()` at line 128 is not released after line 136.'}
"""

## Mount Drive and Define Adapter Paths

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
path_codefixer = "/content/drive/MyDrive/LJMU_models/CodeLLama_34b_codefixer_ORPOfinetuned_steps400_final_it2"
path_localizer = "/content/drive/MyDrive/LJMU_models/codellama_13b_intruct_hf_localizer_final_iterations450"

## Merge Adapters to Base Models

In [ ]:
base_model_localizer.load_adapter(path_localizer, "localizer")
base_model_localizer.set_adapter('localizer')
base_model_localizer.enable_adapters()

Loading adapter weights from /content/drive/MyDrive/LJMU_models/codellama_13b_intruct_hf_localizer_final_iterations450 led to missing keys in the model: model.layers.0.self_attn.q_proj.lora_A.localizer.weight, model.layers.0.self_attn.q_proj.lora_B.localizer.weight, model.layers.0.self_attn.k_proj.lora_A.localizer.weight, model.layers.0.self_attn.k_proj.lora_B.localizer.weight, model.layers.0.self_attn.v_proj.lora_A.localizer.weight, model.layers.0.self_attn.v_proj.lora_B.localizer.weight, model.layers.0.self_attn.o_proj.lora_A.localizer.weight, model.layers.0.self_attn.o_proj.lora_B.localizer.weight, model.layers.0.mlp.gate_proj.lora_A.localizer.weight, model.layers.0.mlp.gate_proj.lora_B.localizer.weight, model.layers.0.mlp.up_proj.lora_A.localizer.weight, model.layers.0.mlp.up_proj.lora_B.localizer.weight, model.layers.0.mlp.down_proj.lora_A.localizer.weight, model.layers.0.mlp.down_proj.lora_B.localizer.weight, model.layers.1.self_attn.q_proj.lora_A.localizer.weight, model.layers.1

In [ ]:
adapter_model_codefixer = base_model_codefixer.load_adapter(path_codefixer+, "codefixer")
adapter_model_codefixer.set_adapter('codefixer')
adapter_model_codefixer.enable_adapters()


Loading adapter weights from /content/drive/MyDrive/LJMU_models/CodeLLama_34b_codefixer_ORPOfinetuned_steps400_final_it2 led to missing keys in the model: model.layers.0.self_attn.q_proj.lora_A.localizer.weight, model.layers.0.self_attn.q_proj.lora_B.localizer.weight, model.layers.0.self_attn.k_proj.lora_A.localizer.weight, model.layers.0.self_attn.k_proj.lora_B.localizer.weight, model.layers.0.self_attn.v_proj.lora_A.localizer.weight, model.layers.0.self_attn.v_proj.lora_B.localizer.weight, model.layers.0.self_attn.o_proj.lora_A.localizer.weight, model.layers.0.self_attn.o_proj.lora_B.localizer.weight, model.layers.0.mlp.gate_proj.lora_A.localizer.weight, model.layers.0.mlp.gate_proj.lora_B.localizer.weight, model.layers.0.mlp.up_proj.lora_A.localizer.weight, model.layers.0.mlp.up_proj.lora_B.localizer.weight, model.layers.0.mlp.down_proj.lora_A.localizer.weight, model.layers.0.mlp.down_proj.lora_B.localizer.weight, model.layers.1.self_attn.q_proj.lora_A.localizer.weight, model.layer

## Define Agents, Roles and Tasks

### Define PEFT Model Client Class

In [ ]:
class PEFTModelClient:
  """
    Client for interacting with a Parameter-Efficient Fine-Tuning (PEFT) model.

    This class provides a convenient interface for generating text using a PEFT model,
    handling various input formats and configurations.

    Attributes:
        model (transformers.PreTrainedModel): The loaded PEFT model.
        tokenizer (transformers.PreTrainedTokenizer): The tokenizer associated with the model.
        device (str): The device to run the model on (e.g., "cuda" or "cpu").
        max_new_tokens (int): The maximum number of new tokens to generate.

    """
    def __init__(self, config, loaded_model, tokenizer, **kwargs):
      """
        Initializes the PEFTModelClient.

        Args:
            config (dict): A dictionary containing configuration parameters.
                           It should have a "device" key (optional) and a "params" dictionary
                           (optional) which may contain "max_new_tokens" (optional).
            loaded_model (transformers.PreTrainedModel): The pre-trained PEFT model loaded using Transformers.
            tokenizer (transformers.PreTrainedTokenizer): The tokenizer corresponding to the loaded model.
            **kwargs: Additional keyword arguments.

        """
        self.model          = loaded_model
        self.tokenizer      = tokenizer
        self.device         = config.get("device", "cuda")
        self.max_new_tokens = config.get("params", {}).get("max_new_tokens", 256)

        self.tokenizer.pad_token_id = self.tokenizer.eos_token_id

    def create(self, params):
        """
        Generates text based on the provided input parameters.

        Args:
            params (dict): A dictionary containing the input parameters for text generation.
                           It must have a "messages" key, which can be a string or a list of messages.
                           Each message in the list can be a string or a dictionary/object with a "content" attribute.

        Returns:
            SimpleNamespace: An object containing the generated response.
                             The response has a 'choices' attribute, which is a list of SimpleNamespace objects.
                             Each choice has a 'message' attribute (another SimpleNamespace) with the generated
                             text in its 'content' attribute. The response also includes the 'model' name.

        Raises:
            ValueError: If the "messages" in the `params` dictionary is not a string or a list,
                        or if a message in the list is not a string, dictionary with "content",
                        or an object with a "content" attribute.
        """

        raw = params["messages"]
        if isinstance(raw, str):
            prompt = raw
        elif isinstance(raw, list):
            parts = []
            for m in raw:
                if isinstance(m, str):
                    parts.append(m)
                elif isinstance(m, dict) and "content" in m:
                    parts.append(m["content"])
                elif hasattr(m, "content"):
                    parts.append(m.content)
                else:
                    raise ValueError(f"Unrecognized message type: {type(m)}")
            prompt = "\n".join(parts)
        else:
            raise ValueError(f"Expected messages to be str or list, got {type(raw)}")

        if not prompt.rstrip().endswith("### Response:"):
            prompt = prompt.rstrip() + "\n### Response:"

        batch = self.tokenizer([prompt], return_tensors="pt")
        batch = {k: v.to(self.device) for k, v in batch.items()}

        outputs = self.model.generate(
            input_ids      = batch["input_ids"],
            attention_mask = batch.get("attention_mask"),
            max_new_tokens = self.max_new_tokens,
            eos_token_id   = self.tokenizer.eos_token_id,
            pad_token_id   = self.tokenizer.pad_token_id,
            do_sample      = True,
            temperature    = 0.3,
            top_p          = 0.9,
        )

        gen_tokens = outputs[0, batch["input_ids"].shape[-1]:]
        text = self.tokenizer.decode(gen_tokens, skip_special_tokens=True)

        response = SimpleNamespace(
            choices=[SimpleNamespace(message=SimpleNamespace(content=text))],
            model=self.model.config._name_or_path
        )
        return response

    def message_retrieval(self, response):
      """
        Extracts the content of the generated messages from the response.

        Args:
            response (SimpleNamespace): The response object returned by the `create` method.

        Returns:
            list[str]: A list of strings, where each string is the content of a generated message.
        """
        return [c.message.content for c in response.choices]

    def cost(self, response):
        return 0.0

    @staticmethod
    def get_usage(response):
        return {}


### Register Custom LLMs to Agents

In [ ]:
custom_localizer_config = [{
    "model": "localizer-model",
    "model_client_cls": "PEFTModelClient"
}]

localizer_agent = AssistantAgent(
    name="bug_localizer",
    llm_config={"config_list": custom_localizer_config}
)

localizer_agent.register_model_client(
    model_client_cls=PEFTModelClient,
    loaded_model=base_model_localizer,
    tokenizer=base_tokenizer_localizer
)

custom_codefixer_config = [{
    "model": "codefixer-model",
    "model_client_cls": "PEFTModelClient"
}]

codefixer_agent = AssistantAgent(
    name="bug_fixer",
    llm_config={"config_list": custom_codefixer_config}
)

codefixer_agent.register_model_client(
    model_client_cls=PEFTModelClient,
    loaded_model=adapter_model_codefixer,
    tokenizer=base_tokenizer_codefixer
)

In [ ]:
llm_judge_config = [{
    "model": "llm-judge-model",
    "model_client_cls": "PEFTModelClient"
}]

llm_judge_config = {
    "name": "LLM-Judge",
    "llm_config": {
        "config_list": llm_judge_config,
        "temperature": 0,
    },
    "system_message": """You are an expert Java code analysis assistant.
        Given the original Java code, a description of the bug, and the proposed fixed code,
        you will analyze whether the fixed code correctly addresses the bug.  Provide a detailed
        analysis of the changes, and then state whether the bug is fixed.  Also provide a
        confidence score (0-100) indicating how likely the fixed code is correct.

        Your response should be structured as follows:

        Analysis: <detailed analysis of the code changes and their impact>
        Bug Fixed: <Yes/No>
        Confidence Score: <0-100>
        Feedback to Localizer: <Feedback for the Bug_Localizer, if any>
        Feedback to Fixer: <Feedback for the Code_Fixer, if any>
        If you cannot compare the code, respond with "CANNOT COMPARE".""",

}
llm_judge_agent = autogen.AssistantAgent(**llm_judge_config)

llm_judge_agent.register_model_client(
    model_client_cls=PEFTModelClient,
    loaded_model=base_model_codefixer,
    tokenizer=base_tokenizer_codefixer
)

[autogen.oai.client: 05-05 10:56:01] {871} INFO - Detected custom model client in config: PEFTModelClient, model client can not be used until register_model_client is called.


INFO:autogen.oai.client:Detected custom model client in config: PEFTModelClient, model client can not be used until register_model_client is called.


# Define The Agent Tasks/Prompts

In [ ]:
localizer_prompt = """
Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request, do not provide explanations just the answer

### Instruction:
You are a JAVA bug localizer agent. Given an input JAVA code containing some bug, and the information about the bug,
your task is to localize the line or lines of code which may contain the bug, and return only the localized patch of code
as output, do not provide any further explanation.

### Input:
#### JAVA Method:
{}

#### Bug Information:
{}

### Response:"""


fixer_prompt = """
Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request, do not provide explanations just the answer

### Instruction:
You are an expert JAVA coder. Given an input buggy method code written in JAVA, information about the bug, and the part of the code containing the probably bug (localized code containing bug), your task is to
analyse the inputs and fix the buggy method and return the fixed method in JAVA. Only return the fixed code, no further explanation required.

### Input:
#### JAVA Method containing bug:
{}

#### Bug Information:
{}

#### Localized Part of the Code Containing Bug:
{}

### Response:"""

## Define The Enhanced Prompts With Judge Feedback For Further Iterations

In [ ]:
localizer_prompt_with_feedback = """
Below is an instruction that describes a task, paired
with an input that provides further context. Write a response that appropriately completes the request, do not provide explanations just the answer

### Instruction:
You are a JAVA bug localizer agent. Given an input JAVA
code containing some bug, and the information about the bug,
your task is to localize the line or lines of code which may contain the bug, and return only the localized patch of code
as output, do not provide any further explanation. You also have acces to feedback provided by an llm judge model, use that to
frame your output as well.

### Input:
#### JAVA Method:
{}

#### Bug Information:
{}

#### Feedback by a judge agent:
{}

### Response:"""

fixer_prompt_with_feedback = """
Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request, do not provide explanations just the answer

### Instruction:
You are an expert JAVA coder. Given an input buggy method code written in JAVA, information about the bug, and the part of the code containing the probably bug (localized code containing bug), your task is to
analyse the inputs and fix the buggy method and return the fixed method in JAVA. Only return the fixed code, no further explanation required.
You also have acces to feedback provided by an llm judge model, use that to
frame your output as well.

### Input:
#### JAVA Method containing bug:
{}

#### Bug Information:
{}

#### Localized Part of the Code Containing Bug:
{}

#### Feedback by a judge agent:
{}

### Response:"""


In [ ]:
llm_judge_prompt = """
Here is the original Java code containing bug:
```java
{}
```
Bug information: {}

Localized Bug generated by the Localizer Model:
{}

Fixed code generated by the Codefixer Model:
```java
{}
```
Compare the fixed code with the original code and provide an analysis.
"""

In [ ]:
user_proxy = autogen.UserProxyAgent(
    name="User_Proxy",
    human_input_mode="NEVER",
    max_consecutive_auto_reply=5,
    is_termination_msg=lambda x: x.get("content", "") and x.get("content", "").rstrip().endswith("TERMINATE"),
    code_execution_config=False
)

# Define The Workflow

In [ ]:
def fix_bug_workflow(original_code, bug_report, max_iterations=3):
    """
    Orchestrates the bug fixing process with a feedback loop.

    Args:
        original_code (str): The original Java code with the bug.
        bug_report (str): A description of the bug.
        max_iterations (int): Maximum number of iterations for the feedback loop.
    """

    iteration = 0
    localized_code = ""
    fixed_code = ""
    comparer_response = ""
    localizer_feedback= ""
    codefixer_feedback = ""
    while iteration < max_iterations:
        iteration += 1
        print(f"\n--- Iteration {iteration} ---")

        if iteration == 1:
            user_proxy.send(localizer_prompt.format(buggy_code, bug_info), localizer_agent)

            localizer_response = localizer_agent.generate_reply(sender = user_proxy)
        else:
            if feedback_localizer != "":
              user_proxy.send(localizer_prompt_with_feedback.format(buggy_code, bug_info, feedback_localizer), localizer_agent)

              localizer_response = localizer_agent.generate_reply(sender = user_proxy)


        print(f"LOCALIZER RESPONSE AT ITERATION {iteration} : \n{localizer_response}")
        if "CANNOT LOCALIZE" in localizer_response:
            print("Bug Localizer failed to localize the bug.")
            return

        if iteration == 1:
          localizer_agent.send(
              message=fixer_prompt.format(buggy_code, bug_info, localizer_response),
              recipient=codefixer_agent,
          )

          fixer_response = codefixer_agent.generate_reply(sender = localizer_agent)

        else:
          if feedback_fixer != "":
              localizer_agent.send(
                  message=fixer_prompt_with_feedback.format(buggy_code, bug_info, localizer_response, feedback_fixer),
                  recipient=codefixer_agent,
              )

              fixer_response = codefixer_agent.generate_reply(sender = localizer_agent)



        print(f"CODEFIXER RESPONSE AT ITERATION {iteration} : \n{fixer_response}")
        if "CANNOT FIX" in fixer_response:
            print("Code Fixer failed to fix the bug.")
            return

        codefixer_agent.send(llm_judge_prompt.format(buggy_code, bug_info, localizer_response, fixer_response), llm_judge_agent)

        judge_response = llm_judge_agent.generate_reply(sender = codefixer_agent)

        print("Code Comparer's Analysis:\n", judge_response)

        if "Bug Fixed: Yes" in judge_response:
            print("\nBug successfully fixed!")
            break

        feedback_localizer = ""
        feedback_fixer = ""
        for line in judge_response.split('\n'):
            if "Feedback to Localizer:" in line:
                feedback_localizer = line.split("Feedback to Localizer:")[1].strip()
            if "Feedback to Fixer:" in line:
                feedback_fixer = line.split("Feedback to Fixer:")[1].strip()

        if "bug fixed: yes" in judge_response.lower():
            print("\nBug successfully fixed!")
            return fixer_response

    print("\nMax iterations reached. Bug not fully fixed.")
    return fixer_response + "\n\n" + judge_response



# Run AutoGen Agent For Sample Input

In [ ]:
response = fix_bug_workflow(buggy_code, bug_info, max_iterations=2)


print(response)


--- Iteration 1 ---
User_Proxy (to bug_localizer):


Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request, do not provide explanations just the answer

### Instruction:
You are a JAVA bug localizer agent. Given an input JAVA code containing some bug, and the information about the bug,
your task is to localize the line or lines of code which may contain the bug, and return only the localized patch of code
as output, do not provide any further explanation.

### Input:
#### JAVA Method:

public String readLine() throws IOException {
	// with these functions, we consider a line as always terminated by CRLF
	serverByteBuffer sb = new serverByteBuffer();
	int c;
	while (true) {
	    c = read();
	    if (c < 0) {
		if (sb.length() == 0) return null; else return sb.toString();
	    }
	    if (c == cr) continue;
	    if (c == lf) return sb.toString();
	    sb.append((byte) c);
	}
    }


###

In [ ]:
print(response)

```java
public String readLine() throws IOException {
	// with these functions, we consider a line as always terminated by CRLF
	try (serverByteBuffer sb = new serverByteBuffer()) {
		int c;
		while (true) {
		    c = read();
		    if (c < 0) {
			if (sb.length() == 0) return null; else return sb.toString();
		    }
		    if (c == cr) continue;
		    if (c == lf) return sb.toString();
		    sb.append((byte) c);
		}
	}
}
```
TERMINATE

Analysis: The original code creates an instance of `serverByteBuffer` without any mechanism to release it, which leads to a resource leak. The fixed code addresses this issue by using a try-with-resources statement, which ensures that the `serverByteBuffer` instance is automatically closed when the try block is exited, either normally or due to an exception. This change effectively manages the resource and prevents the leak. The try-with-resources statement is a best practice in Java for managing resources that implement the `AutoCloseable` interface, whi